<a href="https://colab.research.google.com/github/JonasVerbickas/test-jupyter/blob/main/NLP_CW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import time
import nltk
import string

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
porter = nltk.PorterStemmer()

In [5]:
class StringWithDocId:
  def __init__(self, string, doc_id):
    self.string = string
    self.doc_id = doc_id
  
  def __lt__(token_with_doc_A, token_with_doc_B):
    if token_with_doc_A.string == token_with_doc_B.string:
      return token_with_doc_A.doc_id < token_with_doc_B.doc_id
    else:
      return token_with_doc_A.string < token_with_doc_B.string
    
  def __str__(self):
    return f"{self.string}: {self.doc_id}"

  def __repr__(self):
    return f"{self.string}: {self.doc_id}"

In [9]:
STOP_WORDS = ["the", "is", "am", "are", "will", "was", "were", "!", ".", "?"] # this shouldn't be hard coded
WINDOW_SIZE = 5

class InvertedIndex:
    """
    Construct Inverted Index
    """
    def __init__(self):
        self.inverted_index = {}

    def __getitem__(self, key):
        return self.inverted_index[key]
      
    def keys(self):
      return self.inverted_index.keys()
        
    def read_data(self, path: str) -> list:
        """
        Read files from a directory and then append the data of each file into a list.
        """
        output = []
        for file in os.listdir(path):
          with open(os.path.join(path, file), 'r') as f:
            if file[-3:] != 'txt':
              print(file, "will be skipped")
              continue
            output.append(StringWithDocId(f.read(), file))
        print(len(output))
        return output

    def process_document(self, document: str) -> list:
        """
        pre-process a document and return a list of its terms
        str->list"""
        # 1. Wikipedia hyperlinks should be removed
        # 2. true-case folding
        # 3. stop-words
        # 4. Use multi-word character/location names from .csv files
        document_without_punctuation = document#.translate(None, string.punctuation)
        split_into_words = nltk.tokenize.wordpunct_tokenize(document_without_punctuation)
        output = []
        for word in split_into_words:
          as_lowercase = word.lower()
          if as_lowercase in STOP_WORDS:
            continue
          output.append(as_lowercase)
        return output
    
    def index_corpus(self, documents: list) -> None:
        """
        index given documents
        list->None"""
        starting_time = time.perf_counter()
        token_list = []
        # 1. Generate token sequence
        for doc in documents:
          curr_doc_id = doc.doc_id
          processed_string = self.process_document(doc.string)
          for token in processed_string:
            token = porter.stem(token)
            token_with_doc_obj = StringWithDocId(token, curr_doc_id)
            token_list.append(token_with_doc_obj)
        # 2. Sort
        sorted_token_list = sorted(token_list)
        print("First 10 of sorted_token_list:", sorted_token_list[:10])
        # 3. Convert into dictionary of postings
        for token_with_doc_id in sorted_token_list:
          if token_with_doc_id.string in self.inverted_index:
            if not token_with_doc_id.doc_id in self.inverted_index[token_with_doc_id.string]:
              self.inverted_index[token_with_doc_id.string].append(token_with_doc_id.doc_id)
          else:
            self.inverted_index[token_with_doc_id.string] = [token_with_doc_id.doc_id]
        # Print out some details about the dataset
        total_time_taken = round(time.perf_counter() - starting_time, 4)
        print(f"It took: {total_time_taken} seconds to index the whole corpus.")
        print(f"It has {len(self.inverted_index)} entries in total.")
      
    def dump(self, examples: list) -> None:
        """
        provide a dump function to show index entries for a given set of terms        
        """
        for e in examples:
          processed_e = self.process_document(e)
          print(f'Match for {e}: {self.inverted_index[processed_e]}')

     
    def proximity_search(self, term1: str, term2: str, window_size: int = 3) -> dict:
        """
        1) check whether given two terms appear within a window
        2) calculate the number of their co-existance in a document
        3) add the document id and the number of matches into a dict
        return the dict"""
        documents_containing_both_terms = []
        term1_i = 0
        term2_i = 0
        while term1_i < len(self[term1]) and term2_i < len(self[term2]):
          if self[term1][term1_i] == self[term2][term2_i]:
            documents_containing_both_terms.append(self[term1][term1_i])
            term1_i += 1
            term2_i += 1
          else:
            if self[term1][term1_i] < self[term2][term2_i]:
              term1_i += 1
            else:
              term2_i += 1
        return documents_containing_both_terms
    
    

In [10]:
def main():
    "main call function"
    index = InvertedIndex() # initilaise the index
    corpus = index.read_data('/content/drive/MyDrive/Colab Notebooks/Simpsons2022') # specify the directory path in which files are located
    index.index_corpus(corpus) # index documents/corpus
    
    search_term = input("Enter your query: ") # insert a query
    # write a demo to check entered search terms against the inverted index
        # 1) len(search _term) == one --> return the following: 
            # a) the number of documents in which a term appears.
            # b) all document ids in which a term appears.
    split_into_words = search_term.split(" ")
    if len(split_into_words) == 1:
      print(split_into_words)
      if split_into_words[0] in index.keys():
        documents_matching_the_query = index[split_into_words[0]]
        print(f"The term appears in *{len(documents_matching_the_query)}* documents")
        print("Document IDs are:", documents_matching_the_query)
      else:
        print("Index does not contain the value")

        # 2) len(search_term) == 2 --> return the following: 
            # a) the number of documents in which the entered terms appear within a pre-defined window.
            # b) all document ids in which the terms appear within that window.
    if len(split_into_words) == 2:
      documents_matching_the_query = index.proximity_search(*split_into_words)
      print(f"The combination of these two term appears in *{len(documents_matching_the_query)}* documents")
      print("Document IDs are:", documents_matching_the_query)

    return index
    
index = main()

simpsons_characters - row.csv will be skipped
simpsons_locations row.csv will be skipped
118
First 10 of sorted_token_list: [!": 3.18.txt, !": 3.19.txt, !": 3.2.txt, !": 3.22.txt, !": 3.24.txt, !": 3.7.txt, !": 4.10.txt, !": 4.14.txt, !": 4.15.txt, !": 4.16.txt]
It took: 8.2184 seconds to index the whole corpus.
It has 10115 entries in total.


KeyboardInterrupt: ignored